## Purpose
The purpose of this notebook to load in all the raw videos and images we have in our dataset. Then convert all images to the .png format. Additionally, the videos will be split into frames that will be saved as .png images. Finally, all images that are resized and converted to pngs will be passed into easyOCR reader that will remove most of the text contained in the images.

The following cell imports the needed libraries

In [19]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import imagesize
import csv
from PIL import Image
import easyocr
import shutil

The following cell creates the directory where all the png images will be saved.

In [2]:
try:
    if not os.path.exists('png_images'):
        os.makedirs('png_images')
except OSError:
    print('Error creating data directory.')

The following cell creates the directory where the frames (images) from the videos will be saved.

In [3]:
try:
    if not os.path.exists('video_images'):
        os.makedirs('video_images')
except OSError:
    print('Error creating data directory.')

The following cell creates the directory where all the masked png images will be saved.

In [4]:
try:
    if not os.path.exists('masked_images'):
        os.makedirs('masked_images')
except OSError:
    print('Error creating data directory.')

The following cell contains the function used for converting a video to png images.

In [5]:
def video_to_image(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        file_path = os.path.join(input_dir, filename)  # Correct file path
        capture = cv2.VideoCapture(file_path)
        
        images_created = 0
        currentFrame = 0
        frameSkip = 20 # saves every 20 frames
        
        try: 
            while True:
                ret, frame = capture.read()

                # Break the loop if no frame is captured
                if not ret:
                    break

                # Only save every 'frameSkip'-th frame
                if currentFrame % frameSkip == 0:

                    # Saving the frame
                    name = f'{output_dir}/{filename.split("/")[-1]}_frame{currentFrame}.png'

                    cv2.imwrite(name, frame)
                    images_created += 1

                currentFrame += 1
        except Exception as e:
            print(f"An error occurred: {e}")

        print(f'Images created for {filename}: {images_created}')
        capture.release()
    print("DONE CONVERTING IMAGES TO FRAMES")

The following function iterates through all images in a directory, converts them to a png if they are not already, then resizes them to a certain size for the model.

In [6]:
def resize_images(input_dir, output_dir, size=(256, 256)): # REPLACE SIZE WITH THE MODELS SPECIFICED SIZE
    """Resizes all images in input_dir to the specified size and saves them as PNG in output_dir."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)
        try:
            with Image.open(img_path) as img:
                # Convert to PNG if not already PNG
                if not filename.lower().endswith(".png"):
                    filename = os.path.splitext(filename)[0] + ".png"
                
                img_resized = img.resize(size, Image.Resampling.LANCZOS)
                
                # Save in PNG format
                img_resized.save(os.path.join(output_dir, filename), format="PNG")
        except Exception as e:
            print(f"Skipping {filename}: {e}")

Loops through all the videos in the raw videos directory and creates png images from frames of them. Then it resizes all the images and stores them in the png images directory.

In [7]:
video_to_image("/data/ai_club/team_13_2024-25/VIPR/Data/Raw Videos", "/home/schuberte/VIPR/video_images")
resize_images("/home/schuberte/VIPR/video_images", "/home/schuberte/VIPR/png_images")

Images created for media8.mp4: 14
Images created for media3.MP4: 3
Images created for media2.mp4: 5
Images created for media10.mp4: 6
Images created for media6.mp4: 5
Images created for media4.MP4: 3
Images created for media9.mp4: 12
DONE CONVERTING IMAGES TO FRAMES
Skipping .ipynb_checkpoints: [Errno 21] Is a directory: '/home/schuberte/VIPR/video_images/.ipynb_checkpoints'


Loops through all the images in the raw images directory, resizes and converts them to png images (if they're not already).

In [8]:
resize_images("/data/ai_club/team_13_2024-25/VIPR/Data/Raw Images", "/home/schuberte/VIPR/png_images")

Loops through all the images in the ambiguous directory, resizes and converts them to png images (if they're not already).

In [9]:
resize_images("/data/ai_club/team_13_2024-25/VIPR/Data/dr_sebelik_data/Images/Ambiguous", "/home/schuberte/VIPR/png_images")

Loops through all the images in the healthy (images) directory, resizes and converts them to png images (if they're not already).

In [10]:
resize_images("/data/ai_club/team_13_2024-25/VIPR/Data/dr_sebelik_data/Images/Healthy", "/home/schuberte/VIPR/png_images")

Skipping .ipynb_checkpoints: [Errno 21] Is a directory: '/data/ai_club/team_13_2024-25/VIPR/Data/dr_sebelik_data/Images/Healthy/.ipynb_checkpoints'


This will loop through all the images in the paralyzed (images) directory, resizes and converts them to png images (if they're not already).

In [ ]:
## This is current empty. Need to have Will go through "Ambiguous" directory to determine which ones are paralyzed, if any.
# resize_images("/data/ai_club/team_13_2024-25/VIPR/Data/dr_sebelik_data/Images/Paralyzed", "/home/schuberte/VIPR/png_images")

## Removing Text from Ultrasound Images

The following code reads in the image (in BGR format) then converts it to RGB for outputting on the screen.

In [16]:
def remove_text(input_dir, output_dir):
    # Initialize easyOCR reader
    reader = easyocr.Reader(['en'], gpu=True)

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        img_path = os.path.join(input_dir, filename)

        # Read the image
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if image is None:
            print(f"Skipping {filename}, could not read the image.")
            continue
        
        # Detect text using easyOCR
        results = reader.readtext(image)

        # Create a blank mask with the same size as the image
        mask = np.zeros(image.shape, dtype=np.uint8)

        # Loop through the bounding boxes and fill the areas where text is detected
        for (bbox, text, prob) in results:
            (top_left, top_right, bottom_right, bottom_left) = bbox
            top_left = tuple(map(int, top_left))
            bottom_right = tuple(map(int, bottom_right))

            # Fill the detected text area with white color (or any color that matches the background)
            cv2.rectangle(mask, top_left, bottom_right, (255, 255, 255), thickness=cv2.FILLED)

        # Invert the mask (so text area is black and the rest is white)
        mask_inv = cv2.bitwise_not(mask)

        # Use the mask to remove the text from the image by applying it to the original image
        image_no_text = cv2.bitwise_and(image, mask_inv)
        
        # Save the processed image
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, image_no_text)

        print(f"Processed and saved: {output_path}")

This following cell loops through all the images in the png_images directory (that are of type .png and are resized to the specified size), and applys a mask to the unwanted text, then saved each image in the masked_image directory.

In [17]:
remove_text("/home/schuberte/VIPR/png_images", "/home/schuberte/VIPR/masked_images")

Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame0.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame20.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame40.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame60.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame80.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame100.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame120.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame140.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame160.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame180.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame200.png
Processed and saved: /home/schuberte/VIPR/masked_images/media8.mp4_frame220.png
Processed and saved: /home/schuberte/VIPR/mask

Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_39.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_38.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_36.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_35.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_33.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_32.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_30.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_27.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_24.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_21.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_20.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_18.png
Processed and saved: /home/schuberte/VIPR/masked_images/croppedImage_17.png
Processed an

## Saving Masked Ultrasound Images to Storage

In [20]:
dest = shutil.move("/home/schuberte/VIPR/masked_images", "/data/ai_club/team_13_2024-25/VIPR/")
print(f"Moved the directory to the following location: {dest}")

Moved the directory to the following location: /data/ai_club/team_13_2024-25/VIPR/masked_images
